In [1]:
import requests
import json
from Resources.API_Key import api_key
import pandas as pd

In [2]:
# Using the API key to access BLS data   
url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"

series_id_lp = "PRS85006092"

payload = {
    "seriesid": [series_id_lp],
    "startyear": "2018",
    "endyear": "2024",
    "registrationkey": api_key
}

response = requests.post(url, json=payload)

if response.status_code == 200:
    data = response.json()
    print(json.dumps(data, indent=4))
else:
    print(f"Error {response.status_code}")
    

{
    "status": "REQUEST_SUCCEEDED",
    "responseTime": 164,
    "message": [],
    "Results": {
        "series": [
            {
                "seriesID": "PRS85006092",
                "data": [
                    {
                        "year": "2024",
                        "period": "Q03",
                        "periodName": "3rd Quarter",
                        "latest": "true",
                        "value": "2.2",
                        "footnotes": [
                            {}
                        ]
                    },
                    {
                        "year": "2024",
                        "period": "Q02",
                        "periodName": "2nd Quarter",
                        "value": "2.1",
                        "footnotes": [
                            {
                                "code": "R",
                                "text": "revised"
                            }
                        ]
                    },
   

In [3]:
# Converting the data into a dataframe
series_data = data['Results']['series'][0]['data']
labor_productivity_df = pd.DataFrame(series_data)

# Formatting the data
quarter_mapping = {
    "Q01": "Q1: Jan 1 - Mar 31",
    "Q02": "Q2: Apr 1 - Jun 30",
    "Q03": "Q3: Jul 1 - Sep 30",
    "Q04": "Q4: Oct 1 - Dec 31"
}

# Extra formatting for the data that came from the API request
labor_productivity_df['quarter'] = labor_productivity_df['period'].map(quarter_mapping)
labor_productivity_df['year'] = pd.to_numeric(labor_productivity_df['year'])
labor_productivity_df['percent_change'] = pd.to_numeric(labor_productivity_df['value'])
labor_productivity_df = labor_productivity_df[['year', 'quarter', 'percent_change']]
labor_productivity_df = labor_productivity_df.sort_values(by=['year', 'quarter']).reset_index(drop=True)

In [4]:
# Reading in the Excel file that was used for this data frame
file_path = "Resources/unit_labor_costs.xlsx"
sheet_name = "BLS Data Series"
unit_labor_costs_df = pd.read_excel(file_path)
unit_labor_costs_df = unit_labor_costs_df.melt(id_vars =["Year"],
                                               var_name="quarter",
                                               value_name="percent_change")

#Formatting the data so that it matches the previous data frame
quarter_mapping2 = {
    "Qtr1": "Q1: Jan 1 - Mar 31",
    "Qtr2": "Q2: Apr 1 - Jun 30",
    "Qtr3": "Q3: Jul 1 - Sep 30",
    "Qtr4": "Q4: Oct 1 - Dec 31"
}
unit_labor_costs_df["quarter"] = unit_labor_costs_df["quarter"].map(quarter_mapping2)
unit_labor_costs_df = unit_labor_costs_df.sort_values(by=["Year", "quarter"]).reset_index(drop=True)
unit_labor_costs_df = unit_labor_costs_df.dropna(subset=["percent_change"])

In [5]:
# Rinse and repeat for the next excel sheet
file_path2 = "Resources/real_hourly_compensation.xlsx"
sheet_name2 = sheet_name
real_hourly_compensation_df = pd.read_excel(file_path2)
real_hourly_compensation_df = real_hourly_compensation_df.melt(id_vars = ["Year"],
                                                               var_name="quarter",
                                                               value_name="percent_change")

real_hourly_compensation_df["quarter"] = real_hourly_compensation_df["quarter"].map(quarter_mapping2)
real_hourly_compensation_df = real_hourly_compensation_df.sort_values(by=["Year", "quarter"]).reset_index(drop=True)
real_hourly_compensation_df = real_hourly_compensation_df.dropna(subset=["percent_change"])

In [6]:
file_path3 = "Resources/total_factor_productivity.xlsx"
sheet_name3 = sheet_name
total_factor_productivity_df = pd.read_excel(file_path3)

In [7]:
# Labor Productivity DataFrame
labor_productivity_df

,year,quarter,percent_change
0,2018,Q1: Jan 1 - Mar 31,1.5
1,2018,Q2: Apr 1 - Jun 30,0.0
2,2018,Q3: Jul 1 - Sep 30,1.2
3,2018,Q4: Oct 1 - Dec 31,-0.7
4,2019,Q1: Jan 1 - Mar 31,3.4
5,2019,Q2: Apr 1 - Jun 30,2.4
6,2019,Q3: Jul 1 - Sep 30,4.6
7,2019,Q4: Oct 1 - Dec 31,3.8
8,2020,Q1: Jan 1 - Mar 31,-1.1
9,2020,Q2: Apr 1 - Jun 30,20.2


In [8]:
# Unit Labor Costs DataFrame
unit_labor_costs_df

,Year,quarter,percent_change
0,2018,Q1: Jan 1 - Mar 31,1.1
1,2018,Q2: Apr 1 - Jun 30,2.0
2,2018,Q3: Jul 1 - Sep 30,2.8
3,2018,Q4: Oct 1 - Dec 31,2.1
4,2019,Q1: Jan 1 - Mar 31,6.5
5,2019,Q2: Apr 1 - Jun 30,-2.3
6,2019,Q3: Jul 1 - Sep 30,-4.0
7,2019,Q4: Oct 1 - Dec 31,3.4
8,2020,Q1: Jan 1 - Mar 31,11.8
9,2020,Q2: Apr 1 - Jun 30,3.7


In [9]:
# Real Hourly Compensation Data Frame
real_hourly_compensation_df

,Year,quarter,percent_change
0,2018,Q1: Jan 1 - Mar 31,-0.8
1,2018,Q2: Apr 1 - Jun 30,-0.1
2,2018,Q3: Jul 1 - Sep 30,2.2
3,2018,Q4: Oct 1 - Dec 31,-0.2
4,2019,Q1: Jan 1 - Mar 31,8.8
5,2019,Q2: Apr 1 - Jun 30,-2.7
6,2019,Q3: Jul 1 - Sep 30,-1.1
7,2019,Q4: Oct 1 - Dec 31,4.1
8,2020,Q1: Jan 1 - Mar 31,9.2
9,2020,Q2: Apr 1 - Jun 30,28.8


In [10]:
# Total Factor Productivity Data Frame
total_factor_productivity_df

,Year,Annual
0,2018,100.633
1,2019,101.987
2,2020,101.756
3,2021,105.053
4,2022,103.343
5,2023,104.107


In [ ]:
file_path3 = "Resources/labor-productivity-detailed-industries.xlsx"
sheet_name3 = "MachineReadable"
industry_data_df = pd.read_excel(file_path3, sheet_name3)
industry_data_df = industry_data_df[industry_data_df["Year"] >= 2018]
selected_columns = ["Sector", "Industry", "Measure", "Units", "Year", "Value"]
industry_data_df = industry_data_df[selected_columns]
industry_data_df.head(10)

In [12]:
industry_data_filtered = industry_data_df[industry_data_df["Measure"] == "Unit labor costs"]
sector_unit_labor_costs_df = industry_data_filtered.groupby(["Sector", "Year"])["Value"].mean().reset_index()
sector_unit_labor_costs_df = sector_unit_labor_costs_df.rename(columns={"Value": "Unit Labor Costs"})
sector_unit_labor_costs_df.head(10)

,Sector,Year,Unit Labor Costs
0,Accommodation and Food Services,2018,52.688964
1,Accommodation and Food Services,2019,53.426750
2,Accommodation and Food Services,2020,56.465536
3,Accommodation and Food Services,2021,49.246214
4,Accommodation and Food Services,2022,62.352214
5,Accommodation and Food Services,2023,60.194300
6,Administrative and Support and Waste Managemen...,2018,39.146917
7,Administrative and Support and Waste Managemen...,2019,43.705750
8,Administrative and Support and Waste Managemen...,2020,58.409000
9,Administrative and Support and Waste Managemen...,2021,38.089000


In [13]:
first_yr1 = sector_unit_labor_costs_df[sector_unit_labor_costs_df["Year"] == 2018]
last_yr1 = sector_unit_labor_costs_df[sector_unit_labor_costs_df["Year"] == 2023]

merged_df1 = pd.merge(first_yr1, last_yr1, on="Sector", suffixes=("_2018", "_2023"))
merged_df1["Unit Labor Costs % Change"] = ((merged_df1["Unit Labor Costs_2023"] - merged_df1["Unit Labor Costs_2018"]) / merged_df1["Unit Labor Costs_2018"]) * 100
sector_unit_labor_costs_percent_change = merged_df1[["Sector", "Unit Labor Costs_2018", "Unit Labor Costs_2023", "Unit Labor Costs % Change"]]
sector_unit_labor_costs_percent_change

,Sector,Unit Labor Costs_2018,Unit Labor Costs_2023,Unit Labor Costs % Change
0,Accommodation and Food Services,52.688964,60.194300,14.244607
1,Administrative and Support and Waste Managemen...,39.146917,59.396000,51.725870
2,"Arts, Entertainment, and Recreation",48.880150,52.834000,8.088866
3,Finance and Insurance,52.686500,62.949000,19.478424
4,Health Care and Social Assistance,52.323400,55.937500,6.907235
5,Information,53.223045,48.234778,-9.372383
6,Manufacturing,51.330373,67.040074,30.605079
7,Mining,49.175536,61.043821,24.134533
8,Other Services (except Public Administration),55.238071,70.320500,27.304408
9,"Professional, Scientific, and Technical Services",50.766929,61.315000,20.777447


In [14]:
industry_data_filtered2 = industry_data_df[industry_data_df["Measure"] == "Labor productivity"]
sector_labor_productivity_df = industry_data_filtered2.groupby(["Sector", "Year"])["Value"].mean().reset_index()
sector_labor_productivity_df = sector_labor_productivity_df.rename(columns={"Value": "Labor Productivity"})
sector_labor_productivity_df.head(10)

,Sector,Year,Labor Productivity
0,Accommodation and Food Services,2018,49.873857
1,Accommodation and Food Services,2019,50.697214
2,Accommodation and Food Services,2020,49.947857
3,Accommodation and Food Services,2021,68.804679
4,Accommodation and Food Services,2022,54.306179
5,Accommodation and Food Services,2023,60.079600
6,Administrative and Support and Waste Managemen...,2018,65.829250
7,Administrative and Support and Waste Managemen...,2019,65.009333
8,Administrative and Support and Waste Managemen...,2020,55.229583
9,Administrative and Support and Waste Managemen...,2021,88.816750


In [15]:
first_yr2 = sector_labor_productivity_df[sector_labor_productivity_df["Year"] == 2018]
last_yr2 = sector_labor_productivity_df[sector_labor_productivity_df["Year"] == 2023]

merged_df2 = pd.merge(first_yr2, last_yr2, on="Sector", suffixes=("_2018", "_2023"))
merged_df2["Labor Productivity % Change"] = ((merged_df2["Labor Productivity_2023"] - merged_df2["Labor Productivity_2018"]) / merged_df2["Labor Productivity_2018"]) * 100
sector_labor_productivity_percent_change = merged_df2[["Sector", "Labor Productivity_2018", "Labor Productivity_2023", "Labor Productivity % Change"]]
sector_labor_productivity_percent_change

,Sector,Labor Productivity_2018,Labor Productivity_2023,Labor Productivity % Change
0,Accommodation and Food Services,49.873857,60.079600,20.463111
1,Administrative and Support and Waste Managemen...,65.829250,75.261000,14.327598
2,"Arts, Entertainment, and Recreation",53.186000,74.635500,40.329222
3,Finance and Insurance,49.683000,55.393000,11.492865
4,Health Care and Social Assistance,49.785400,59.065000,18.639199
5,Information,52.460682,65.762472,25.355733
6,Manufacturing,50.115217,47.211426,-5.794230
7,Mining,53.408643,59.662000,11.708512
8,Other Services (except Public Administration),49.015679,47.696750,-2.690830
9,"Professional, Scientific, and Technical Services",51.826250,51.830250,0.007718


In [20]:
percent_change_sector_comparison_df = pd.merge(sector_labor_productivity_percent_change, sector_unit_labor_costs_percent_change, on="Sector")
percent_change_sector_comparison_df

,Sector,Labor Productivity_2018,Labor Productivity_2023,Labor Productivity % Change,Unit Labor Costs_2018,Unit Labor Costs_2023,Unit Labor Costs % Change
0,Accommodation and Food Services,49.873857,60.079600,20.463111,52.688964,60.194300,14.244607
1,Administrative and Support and Waste Managemen...,65.829250,75.261000,14.327598,39.146917,59.396000,51.725870
2,"Arts, Entertainment, and Recreation",53.186000,74.635500,40.329222,48.880150,52.834000,8.088866
3,Finance and Insurance,49.683000,55.393000,11.492865,52.686500,62.949000,19.478424
4,Health Care and Social Assistance,49.785400,59.065000,18.639199,52.323400,55.937500,6.907235
5,Information,52.460682,65.762472,25.355733,53.223045,48.234778,-9.372383
6,Manufacturing,50.115217,47.211426,-5.794230,51.330373,67.040074,30.605079
7,Mining,53.408643,59.662000,11.708512,49.175536,61.043821,24.134533
8,Other Services (except Public Administration),49.015679,47.696750,-2.690830,55.238071,70.320500,27.304408
9,"Professional, Scientific, and Technical Services",51.826250,51.830250,0.007718,50.766929,61.315000,20.777447
